---
<center>

  # **Tarea 06**

**Realizado Por:**

   Samuel Huertas Rojas

---
</center>

# 1. 
Compruebe el teorema del virial para el átomo de Hidrógeno a partir de los elementos de matriz
de los operadores de energía cinética y potencial.

El teorema del virial nos dice que:

<center>

$
\braket{T} = - \dfrac{1}{2} \braket{U}
$

</center>

Para comprobar el teorema del virial, vamos a utilizar la representación matricial de cada unos de los operadores de energía cinética y potencial, los cuales se definen como:

* Energía cinética:

<center>

$
\hat{T} = - \dfrac{\hbar^2}{2 \mu} \nabla^2 = - \dfrac{\hbar^2}{2 \mu} \left(\dfrac{d^2}{dr^2} + \dfrac{2}{r} \dfrac{d}{dr} - \dfrac{l(l+1)}{r^2} \right)
$

</center>

* Energía potencial:

<center>

$
\hat{U} = - \dfrac{Z}{r}
$

</center>

In [2]:
import sympy as sp
from sympy.physics.hydrogen import Psi_nlm
from sympy import symbols, simplify, diff

# Constantes (unidades atómicas)
a0 = 1.0  # Radio de Bohr en unidades atómicas
e = 1.0  # Carga del electrón en unidades atómicas
hbar = 1.0  # Constante de Planck reducida en unidades atómicas
me = 1.0  # Masa del electrón en unidades atómicas
mu = 1  # Masa reducid= mM/(m + M)
Z = 1  # Número átomico, número de protones en el núcleo de un átomo
alpha = 1 / 137
c = 1 / alpha

# Definición de los simbolos a utilizar en las coordenadas esfericas
r, theta, phi = symbols("r theta phi", real=True)


# Definición de la función de onda del átomo de hidrogeno
def funcion_onda(n, l, m, Z=1):  # noqa: E741
    """
    Calcula la función radial R_nl(r) del átomo de hidrógeno.

    Parámetros:
    -----------
    n : int
        Número cuántico principal (n >= 1)
    l : int
        Número cuántico orbital (0 <= l < n)
    Z : int
        Número atómico (Z=1 para hidrógeno)
    usar_sympy : bool
        Si True, usa la función incorporada de SymPy
        Si False, implementa la fórmula manualmente

    Retorna:
    --------
    R_nl : expresión simbólica de SymPy
        La función radial en términos de r
    """

    # if l >= n:
    #     raise ValueError(f"l debe ser menor que n. Recibido: n={n}, l={l}")

    # fun = R_nl(n, l, r, Z)

    fun = Psi_nlm(n, l, m, r, phi, theta)

    return simplify(fun)


def L2(f):
    return sp.simplify(
        -(hbar**2)
        * (
            1 / sp.sin(theta) * sp.diff(sp.sin(theta) * sp.diff(f, theta), theta)
            + 1 / sp.sin(theta) ** 2 * sp.diff(f, phi, 2)
        )
    )


# Definición del operador de la energía cinética en coordenadas esfericas
def energia_cinetica(f, l):  # noqa: E741
    E = -(hbar**2) / (2 * mu) * (diff(f, r, 2) + 2 / r * diff(f, r)) + L2(f) / (
        r**2 * 2 * mu
    )
    return E


# Definición del operedaro de la energía potencial
def energia_potencial(f):
    E = -Z / r * f
    return E


# Obtención de los elementos de matriz de los operadores
# Operador energía cinética
def elemento_cinetico(n, l, m):  # noqa: E741
    Psi = funcion_onda(n, l, m)  # noqa: E741
    Psi_cinetico = energia_cinetica(Psi, l)
    Psi_conjugado = sp.conjugate(Psi)

    # Cálculo de la energía esperada
    integrando = Psi_conjugado * Psi_cinetico * r**2 * sp.sin(theta)

    return sp.integrate(
        integrando, (r, 0, sp.oo), (theta, 0, sp.pi), (phi, 0, 2 * sp.pi)
    )


# Operador energía potencial
def elemento_potencial(n, l, m):  # noqa: E741
    Psi = funcion_onda(n, l, m)  # noqa: E741
    elem = energia_potencial(Psi)
    Psi_conjugado = sp.conjugate(Psi)

    # Cálculo de la energía esperada
    integrando = Psi_conjugado * elem * r**2 * sp.sin(theta)

    return sp.integrate(
        integrando, (r, 0, sp.oo), (theta, 0, sp.pi), (phi, 0, 2 * sp.pi)
    )


# Verificación del teorema del virial
print("=" * 70)
print("VERIFICACIÓN DEL TEOREMA DEL VIRIAL EN EL ÁTOMO DE HIDRÓGENO")
print("=" * 70)
print(f"{'Estado':<15} {'⟨T⟩ [u.a]':<15} {'⟨V⟩ [u.a]':<15} {'⟨T⟩ / (-½⟨V⟩) [u.a]':<20}")
print("-" * 80)

# Estados a verificar
n_max = 3
l_max = n_max - 1

estados = []
for n in range(1, n_max + 1):
    for l in range(0, min(n, l_max + 1)):  # noqa: E741
        for m in range(-l, l + 1):
            estados.append((n, l, m))

resultados = []

for n, l, m in estados:  # noqa: E741
    T_calc = elemento_cinetico(n, l, m)
    U_calc = elemento_potencial(n, l, m)

    # Verificación del teorema del virial
    virial_ratio = T_calc / (-0.5 * U_calc)

    resultados.append(
        {
            "state": f"{n}{'spdfgh'[l]}",
            "n": n,
            "l": l,
            "T": T_calc,
            "V": U_calc,
            "virial_ratio": virial_ratio,
        }
    )

    print(
        f"\n({n},{'spdfgh'[l]},{m}) \t\t{T_calc:.4f} \t\t{U_calc:.4f} \t{T_calc / (-0.5 * U_calc):.4f}"
    )
    # print(f" {T_calc:<15.4f} {U_calc:<15.4f} {virial_ratio:<20.4f}")


VERIFICACIÓN DEL TEOREMA DEL VIRIAL EN EL ÁTOMO DE HIDRÓGENO
Estado          ⟨T⟩ [u.a]       ⟨V⟩ [u.a]       ⟨T⟩ / (-½⟨V⟩) [u.a] 
--------------------------------------------------------------------------------

(1,s,0) 		0.5000 		-1.0000 	1.0000

(2,s,0) 		0.1250 		-0.2500 	1.0000

(2,p,-1) 		0.1250 		-0.2500 	1.0000

(2,p,0) 		0.1250 		-0.2500 	1.0000

(2,p,1) 		0.1250 		-0.2500 	1.0000

(3,s,0) 		0.0556 		-0.1111 	1.0000

(3,p,-1) 		0.0556 		-0.1111 	1.0000

(3,p,0) 		0.0556 		-0.1111 	1.0000

(3,p,1) 		0.0556 		-0.1111 	1.0000

(3,d,-2) 		0.0556 		-0.1111 	1.0000

(3,d,-1) 		0.0556 		-0.1111 	1.0000

(3,d,0) 		0.0556 		-0.1111 	1.0000

(3,d,1) 		0.0556 		-0.1111 	1.0000

(3,d,2) 		0.0556 		-0.1111 	1.0000


# 2. 
Encuentre la corrección relativista a la energía cinética calculando el elemento de matriz del
operador energía cinética al cuadrado.

In [3]:
factor_convercion = {"joul": 4.3598e-18, "cm": 219474.6313, "eV": 27.2114}


# Cálculo de la corrección relativista ⟨T²⟩
def correcion_relativista_T2(n, l, m):  # noqa: E741
    """
    Calcula ⟨T²⟩ para la corrección relativista

    Parámetros:
    - n, l, m: números cuánticos
    - unidades: "hartree", "eV", "cm", "joule"
    """
    from sympy import conjugate, lambdify
    from scipy.integrate import nquad

    # print(f"Calculando estado |n={n}, l={l}, m={m}⟩...")
    def T(f):
        T_val = (
            1 / r**2 * diff(r**2 * diff(f, r), r)
            + 1 / (r**2 * sp.sin(theta)) * diff(sp.sin(theta) * diff(f, theta), theta)
            + 1 / (r**2 * sp.sin(theta) ** 2) * diff(f, phi, 2)
        )

        return T_val * -hbar / (2 * mu)

    # Aplicar operador T
    T_Psi = T(Psi_nlm(n, l, m, r, phi, theta))

    # Energia cinetica conjugada
    T_Psi_conjugado = conjugate(T(Psi_nlm(n, l, m, r, phi, theta)))

    # Calcular ⟨T²⟩ = ⟨Ψ|T²|Ψ⟩ = ⟨Ψ T|T Ψ⟩
    integrando = lambdify(
        (r, theta, phi), T_Psi_conjugado * T_Psi * r**2 * sp.sin(theta)
    )

    # Integrar
    resultado, _ = nquad(integrando, [[0, 1e3], [0, sp.pi], [0, 2 * sp.pi]])

    return resultado * (-hbar / (2 * mu * c**2))


# Mostrar los resultados
def print_matriz_compacta(n_max, l_max=None, unidades="cm"):
    """
    Imprime la matriz en formato compacto tipo tabla
    """
    if l_max is None:
        l_max = n_max - 1

    estados = []
    for n in range(1, n_max + 1):
        for l in range(0, min(n, l_max + 1)):  # noqa: E741
            for m in range(-l, l + 1):
                estados.append((n, l, m))

    unidad_str = {"hartree": "E_h²", "cm": "cm⁻¹", "eV": "eV", "joule": "J"}

    print("\nMATRIZ DE CORRECCIÓN RELATIVISTA ⟨T²⟩")
    print(f"Unidades: {unidad_str.get(unidades, unidades)}")
    print("=" * 100)
    print(f"{'Estado':<15} {'n':<5} {'l':<5} {'m':<5} {'⟨T²⟩'}")
    print("-" * 100)

    for n, l, m in estados:  # noqa: E741
        orbital_labels = {0: "s", 1: "p", 2: "d", 3: "f"}
        label = f"{n}{orbital_labels.get(l, f'(l={l})')}"
        estado = f"|{label}, m={m:+d}⟩"

        valor = correcion_relativista_T2(n, l, m)

        print(
            f"{estado:<15} {n:<5} {l:<5} {m:<5} {valor * factor_convercion[unidades]:.6f}"
        )

    print("=" * 100)


print_matriz_compacta(2, unidades="cm")



MATRIZ DE CORRECCIÓN RELATIVISTA ⟨T²⟩
Unidades: cm⁻¹
Estado          n     l     m     ⟨T²⟩
----------------------------------------------------------------------------------------------------
|1s, m=+0⟩      1     0     0     -7.308415
|2s, m=+0⟩      2     0     0     -1.187617


c:\Users\samhu\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_quadpack_py.py:606: ComplexWarning: Casting complex values to real discards the imaginary part
  return _quadpack._qagse(func,a,b,args,full_output,epsabs,epsrel,limit)


|2p, m=-1⟩      2     1     -1    -0.213162
|2p, m=+0⟩      2     1     0     -0.213162
|2p, m=+1⟩      2     1     1     -0.213162


# 3.
Escriba las expresiones analíticas de las correcciones relativistas y compruebe que se obtiene lo
mismo que en el punto anterior.

Las correciones relativistas para el Hamiltoniano van a ser:

$$
\hat{H} = \hat{H}_{0} + \hat{H}_{1} + \hat{H}_{2} + \hat{H}_{3}
$$

Donde:

$$
\hat{H}_{0} = \frac{\hat{p}^{2}}{2m} - \frac{Ze^{2}}{r}
$$

$$
\hat{H}_{1} = -\frac{\hat{p}^{4}}{8m^{3}c^{2}}
$$

$$
\hat{H}_{2} = \xi(r) \hat{L} \cdot \hat{S} = \frac{1}{2m^{2}c^{2}}\frac{1}{r}\frac{\text{d}V}{\text{d}r} \vec{L} \cdot \vec{S}
$$

$$
\hat{H}_{3} = \frac{\pi \hbar^{2}}{2m^{2}c^{2}}Ze^{2}\delta(\vec{r})
$$

Donde $\hat{H}_{1}$ es la correción cinetica relativista, $\hat{H}_{2}$ es el termino pin-orbital y $\hat{H}_{3}$ es el termino de Darwin

Para encontrar el $\Delta E_1$:

$$
\Delta E_1 = -\dfrac{1}{2 m c^2} \braket{\psi_{n,l,m} | \dfrac{p^4}{4 m^2} |\psi_{n,l,m}}
$$

$$
\Delta E_1 = - E_n \dfrac{(Z \alpha)^2}{n^2} \left[ \dfrac{3}{4} - \dfrac{n}{l + 1/2} \right]
$$

Para encontrar $\Delta E_2$:

$$
\Delta E_2  = \braket{\psi_{n,l,m} | 1/2 \xi(r) [\hat{J}^2 - \hat{L}^2 - \hat{S}] |\psi_{n,l,m}}
$$


$$
\Delta E_2 =  -E_n \, \frac{(Z\alpha)^2}{2n l (l + \tfrac{1}{2})(l + 1)} 
\times 
\begin{cases}
l, & \text{para } j = l + \tfrac{1}{2} \\[6pt]
-(l + 1), & \text{para } j = l - \tfrac{1}{2}
\end{cases}
$$

Para encontrar $\Delta E_3$

$$
\Delta E_3 = \dfrac{\pi \hbar^2}{2 m^2 c^2} \dfrac{Z e^2}{4 \pi \epsilon_0} \braket{\psi_{n, 0, 0}|\delta(r)|\psi_{n, 0, 0}}
$$

$$
\Delta E_3 = - E_n \dfrac{(Z \alpha)^2}{n}
$$

Juntando todo:

$$
\Delta E_{nj} = -\frac{1}{2} m c^2 \frac{(Z\alpha)^2}{n^2} \frac{(Z\alpha)^2}{n^2}
\left[ \frac{n}{j + \tfrac{1}{2}} - \frac{3}{4} \right]
= E_n \frac{(Z\alpha)^2}{n^2}
\left[ \frac{n}{j + \tfrac{1}{2}} - \frac{3}{4} \right]
$$

Finalmente: 

$$
E_{nj} = E_n \left[ 1 + \frac{(Z\alpha)^2}{n^2} 
\left( \frac{n}{j + \tfrac{1}{2}} - \frac{3}{4} \right) \right]
$$

Con:

* $
E_n = - \dfrac{1}{2} \mu c^2 \dfrac{(Z \alpha)^2}{n^2}
$

* $\alpha \approx 1/137$
* $c = 1/\alpha$

In [4]:
c = 1 / alpha
factor_convercion = {"joul": 4.3598e-18, "cm": 219474.6313, "eV": 27.2114}


# Definición de En
def En(n):
    return -(1 / 2) * mu * c**2 * ((Z * alpha) ** 2 / n**2)


# Definición de la corrección analitica de la energía cinética
def delta_E1(n, l):  # noqa: E741
    E1 = -En(n) * ((Z * alpha) ** 2 / n**2) * (3 / 4 - n / (l + 1 / 2))

    return E1


def delta_E2(n, l, j):  # noqa: E741
    """
    Calcula la corrección de estructura fina relativista.
    Para l=0, usa una fórmula especial sin singularidad.
    """
    # Energía del nivel n sin corrección
    E_n0 = -En(n)

    # Caso especial: l = 0 (orbital s)
    if l != 0:
        E2 = E_n0 * (Z * alpha) ** 2 / (2 * n * l * (l + 1 / 2) * (l + 1))

        if j == l + 1 / 2:
            return E2 * l
        if j == l - 1 / 2:
            return E2 * -(l + 1)
    else:
        return 0


def delta_E3(n, l):  # noqa: E741
    if l == 0:
        return -En(n) * ((Z * alpha) ** 2 / n)
    else:
        return 0


print("\nMATRIZ DE CORRECCIÓN RELATIVISTA T² (Formato Compacto)")
print("=" * 100)
print(
    f"{'Estado':<15} {'n':<5} {'l':<5} {'m':<5} {'ΔE1 [cm^-1]':<15} {'ΔE2 [cm^-1]':<15} {'ΔE3 [cm^-1]':<15} {'ΔEt [cm^-1]'}"
)
print("-" * 100)

n_max = 3
l_max = n_max - 1

estados = []
for n in range(1, 4):  # puedes cambiar 4 por el número máximo de niveles
    for l in range(0, n):  # noqa: E741
        if l == 0:
            estados.append((n, l, 0.5))
        else:
            estados.append((n, l, l - 0.5))
            estados.append((n, l, l + 0.5))


for n, l, j in estados:  # noqa: E741
    orbital_labels = {0: "s", 1: "p", 2: "d", 3: "f"}
    label = f"{n}{orbital_labels.get(l, f'(l={l})')}"
    estado = f"|{label}⟩"

    valor_E1 = delta_E1(n, l)
    valor_E1_convercion = valor_E1 * factor_convercion["cm"]

    valor_E2 = delta_E2(n, l, j)
    valor_E2_convercion = valor_E2 * factor_convercion["cm"]

    valor_E3 = delta_E3(n, l)
    valor_E3_convercion = valor_E3 * factor_convercion["cm"]

    Et = valor_E1_convercion + valor_E2_convercion + valor_E3_convercion

    print(
        f"{estado:<15} {n:<5} {l:<5} {j:<5} {valor_E1_convercion:.6f} \t{valor_E2_convercion:.6f} \t{valor_E3_convercion:.6f} \t{Et:.6f}"
    )



MATRIZ DE CORRECCIÓN RELATIVISTA T² (Formato Compacto)
Estado          n     l     m     ΔE1 [cm^-1]     ΔE2 [cm^-1]     ΔE3 [cm^-1]     ΔEt [cm^-1]
----------------------------------------------------------------------------------------------------
|1s⟩            1     0     0.5   -7.308415 	0.000000 	5.846732 	-1.461683
|2s⟩            2     0     0.5   -1.187617 	0.000000 	0.730842 	-0.456776
|2p⟩            2     1     0.5   -0.213162 	-0.243614 	0.000000 	-0.456776
|2p⟩            2     1     1.5   -0.213162 	0.121807 	0.000000 	-0.091355
|3s⟩            3     0     0.5   -0.378955 	0.000000 	0.216546 	-0.162409
|3p⟩            3     1     0.5   -0.090227 	-0.072182 	0.000000 	-0.162409
|3p⟩            3     1     1.5   -0.090227 	0.036091 	0.000000 	-0.054136
|3d⟩            3     2     1.5   -0.032482 	-0.021655 	0.000000 	-0.054136
|3d⟩            3     2     2.5   -0.032482 	0.014436 	0.000000 	-0.018045


# 4.
Diagonalice la matriz de espín-órbita para el estado $n=3$, $l=1$ y compruebe que da lo mismo que
la expresión analítica.

Tenemos que encontrar $\hat{L} \cdot \hat{S}$, para encontrar el producto punto vamos a utilizar:

$$
\hat{L} \cdot \hat{S} = \hat{L}_x\hat{S}_x + \hat{L}_y\hat{S}_y + \hat{L}_z\hat{S}_z
$$

Donde $\hat{J_x} = \dfrac{1}{2} (\hat{J_+} + \hat{J_-}) $ y $\hat{J_y} = \dfrac{1}{2i} (\hat{J_+} - \hat{J_-}) $, lo que nos da:

$$
\hat{L} \cdot \hat{S} =  \hat{L}_z\hat{S}_z + \dfrac{1}{2} \left( \hat{L}_+\hat{S}_- + \hat{L}_-\hat{S}_+ \right)
$$

In [5]:
import sympy as sp  # noqa: F811
import numpy as np  # noqa: F811

from sympy.physics.quantum.state import OrthogonalKet
from sympy.physics.quantum import TensorProduct

Ket = OrthogonalKet
hbar = sp.symbols("hbar", real=True)  # noqa: F811

alpha = 1 / 137.036
c = 1 / alpha  # noqa: F811

mu = 1
e = 1  # noqa: F811
Z = 1

n, l, s = 3, 1, 1 / 2  # noqa: E741


# Definición de la matriz Jz
def Jz_autovalores(autoestados):
    j, m = autoestados.label

    return hbar * m


def matriz_Jz(autoestados):
    dim = len(autoestados)
    iden = sp.eye(dim)

    Jz = sp.Matrix([])
    for n in range(dim):
        Jz = Jz.col_insert(n, Jz_autovalores(autoestados[n]) * iden.col(n))

    return Jz


# Definicón de las bases del espacio
base_s = [
    Ket(s, n) for n in np.arange(s, -s - 1, -1)
]  # Recordamos que indexamos las bases de mayor a menor.
base_l = [Ket(l, n) for n in np.arange(l, -l - 1, -1)]

# Creación de la matriz del spin
Sz = matriz_Jz(base_s)
Sz

Matrix([
[0.5*hbar,         0],
[       0, -0.5*hbar]])

In [6]:
Lz = matriz_Jz(base_l)
Lz

Matrix([
[hbar, 0,     0],
[   0, 0,     0],
[   0, 0, -hbar]])

In [7]:
# Producto tensoria entre las matrices
LzSz = TensorProduct(Lz, Sz)
LzSz

Matrix([
[0.5*hbar**2,            0, 0, 0,            0,           0],
[          0, -0.5*hbar**2, 0, 0,            0,           0],
[          0,            0, 0, 0,            0,           0],
[          0,            0, 0, 0,            0,           0],
[          0,            0, 0, 0, -0.5*hbar**2,           0],
[          0,            0, 0, 0,            0, 0.5*hbar**2]])

In [8]:
import functools as ft


def elementos_Jpm(sign, bra, ket):
    j, m = ket.label

    ret = eval(
        f"hbar*sp.sqrt(j*(j + 1) - m*(m {sign} 1))*(bra*Ket(j, m {sign} 1)).doit()"
    )

    return ret


def matriz_Jpm(sign, basis):
    dim = len(basis)

    Jpm = [[0 for i in range(dim)] for j in range(dim)]

    for i in range(dim):
        for j in range(dim):
            Jpm[i][j] = elementos_Jpm(sign, basis[i].dual, basis[j])

    Jpm = sp.Matrix(Jpm)

    return Jpm


Jp = ft.partial(matriz_Jpm, "+")
Jm = ft.partial(matriz_Jpm, "-")

# Creación de las matrices
Sp, Sm = Jp(base_s), Jm(base_s)
Lp, Lm = Jp(base_l), Jm(base_l)


In [9]:
Sp

Matrix([
[0, 1.0*hbar],
[0,        0]])

In [10]:
Lp

Matrix([
[0, sqrt(2)*hbar,            0],
[0,            0, sqrt(2)*hbar],
[0,            0,            0]])

In [11]:
# Producto tensoria entre LpSm
LpSm = TensorProduct(Lp, Sm)
LpSm

Matrix([
[0, 0,                   0, 0,                   0, 0],
[0, 0, 1.0*sqrt(2)*hbar**2, 0,                   0, 0],
[0, 0,                   0, 0,                   0, 0],
[0, 0,                   0, 0, 1.0*sqrt(2)*hbar**2, 0],
[0, 0,                   0, 0,                   0, 0],
[0, 0,                   0, 0,                   0, 0]])

In [12]:
# Producto tensoria entre LmSp
LmSp = TensorProduct(Lm, Sp)
LmSp

Matrix([
[0,                   0, 0,                   0, 0, 0],
[0,                   0, 0,                   0, 0, 0],
[0, 1.0*sqrt(2)*hbar**2, 0,                   0, 0, 0],
[0,                   0, 0,                   0, 0, 0],
[0,                   0, 0, 1.0*sqrt(2)*hbar**2, 0, 0],
[0,                   0, 0,                   0, 0, 0]])

In [13]:
# Producto punto entre el operador L y S
LS = LzSz + 1 / 2 * (LpSm + LmSp)
LS = LS
LS

Matrix([
[0.5*hbar**2,                   0,                   0,                   0,                   0,           0],
[          0,        -0.5*hbar**2, 0.5*sqrt(2)*hbar**2,                   0,                   0,           0],
[          0, 0.5*sqrt(2)*hbar**2,                   0,                   0,                   0,           0],
[          0,                   0,                   0,                   0, 0.5*sqrt(2)*hbar**2,           0],
[          0,                   0,                   0, 0.5*sqrt(2)*hbar**2,        -0.5*hbar**2,           0],
[          0,                   0,                   0,                   0,                   0, 0.5*hbar**2]])

In [14]:
from sympy.physics.hydrogen import R_nl  # noqa: F811

r = sp.symbols("r", real=True)


factor_convercion = {"joul": 4.3598e-18, "cm": 219474.6313, "eV": 27.2114}


def valor_Rnl(fun, n, l=0, conjugate=lambda F: sp.conjugate(F)):  # noqa: E741
    if n <= 0:
        raise ValueError("{} must be positive integer".format(n))

    if not (0 <= l < n):
        raise ValueError("l must be between 0 and {}".format(n - 1))

    Psi_conjugate = conjugate(R_nl(n, l, r))
    operador_Psi = fun(R_nl(n, l, r))

    return sp.integrate(Psi_conjugate * operador_Psi * r**2, (r, 0, sp.oo))


valor_xi = ft.partial(valor_Rnl, lambda F: 1 / (2 * mu**2 * c**2) * Z * e**2 / r**3 * F)

# Matriz del spin-orbital
H2 = valor_xi(n, l) * LS
# Diagonalización de la corrección spin-orbital
H2.eigenvals()


{1.64356030273237e-7*hbar**2: 4, -3.28712060546474e-7*hbar**2: 2}

In [15]:
valor_energia1 = list((H2.eigenvals()).keys())[0].n().subs(hbar, 1)
valor_energia2 = list((H2.eigenvals()).keys())[1].n().subs(hbar, 1)

degeneracion_energia1 = H2.eigenvals()[list((H2.eigenvals()).keys())[0]]
degeneracion_energia2 = H2.eigenvals()[list((H2.eigenvals()).keys())[1]]

print("=" * 70)
print("CORRECCIÓN SPIN-ORBITA A LA ENERGÍA")
print("Átomo de Hidrógeno")
print("=" * 70)
print(f"\nEstado cuántico: |n={n}, ℓ={l}⟩")
print("\n" + "-" * 70)
print(f"E_n1 = {valor_energia1 * factor_convercion['cm']:.4f} cm^-1, degeneración: {degeneracion_energia1}")
print(f"E_n2 = {valor_energia2 * factor_convercion['cm']:.4f} cm^-1, degeneración: {degeneracion_energia2}")


CORRECCIÓN SPIN-ORBITA A LA ENERGÍA
Átomo de Hidrógeno

Estado cuántico: |n=3, ℓ=1⟩

----------------------------------------------------------------------
E_n1 = 0.0361 cm^-1, degeneración: 4
E_n2 = -0.0721 cm^-1, degeneración: 2


# 5. 
Compruebe que los autoestados de la matriz anterior son las autofunciones de $J$ y $Jz$
comparándolos con los coeficientes de Clebsch-Gordan.

In [16]:
# Obtener las autofunciones de la matriz anterior
autofunciones = H2.eigenvects()
# autofunciones

In [17]:
autoestado1 = autofunciones[0][2][0]
autoestado2 = autofunciones[0][2][1]
autoestado3 = autofunciones[1][2][0]
autoestado4 = autofunciones[1][2][1]
autoestado5 = autofunciones[1][2][2]
autoestado6 = autofunciones[1][2][3]

lista_autoestados = [autoestado1, autoestado2, autoestado3, autoestado4, autoestado5, autoestado6]


In [18]:
# Definición de las matrices J y Jz
Jz = hbar * (sp.diag(*[-1, -1, 0, 0, 1, 1]) + 0.5 * sp.diag(*[-1, 1, -1, 1, -1, 1]))
Jz

Matrix([
[-1.5*hbar,         0,         0,        0,        0,        0],
[        0, -0.5*hbar,         0,        0,        0,        0],
[        0,         0, -0.5*hbar,        0,        0,        0],
[        0,         0,         0, 0.5*hbar,        0,        0],
[        0,         0,         0,        0, 0.5*hbar,        0],
[        0,         0,         0,        0,        0, 1.5*hbar]])

In [19]:
# Construcción de las matrices L2 y S2
l = 1  # noqa: E741

valores_l2 = [l * hbar for i in range(2 * (2 * l + 1))]
valores_s2 = [1 / 2 * hbar for i in range(2 * (2 * l + 1))]
matriz_L2 = sp.diag(*valores_l2)
matriz_S2 = sp.diag(*valores_s2)

In [20]:
J2 = matriz_L2 + matriz_S2 + 2 * LS / hbar
J2

Matrix([
[2.5*hbar,                0,                0,                0,                0,        0],
[       0,         0.5*hbar, 1.0*sqrt(2)*hbar,                0,                0,        0],
[       0, 1.0*sqrt(2)*hbar,         1.5*hbar,                0,                0,        0],
[       0,                0,                0,         1.5*hbar, 1.0*sqrt(2)*hbar,        0],
[       0,                0,                0, 1.0*sqrt(2)*hbar,         0.5*hbar,        0],
[       0,                0,                0,                0,                0, 2.5*hbar]])

In [21]:
# normalización de los autoestados
lista_autoestados_normalizados = []
for estado in lista_autoestados:
    suma = 0
    for i in estado:
        suma += i**2
    norma = sp.sqrt(suma)
    estado_normalizado = (1 / norma) * estado
    lista_autoestados_normalizados.append(estado_normalizado)

In [22]:
# Ya tengo la matriz de J2 y Jz, ahora comprobamos que los autoestados son autofunciones
def comprobar_autoestado(A, v):
    A = sp.Matrix(A)
    v = sp.Matrix(v)
    Av = A * v

    # Intenta resolver λ de la ecuación A v = λ v
    # Encuentra un λ consistente
    lambdas = []
    for Ai, vi in zip(Av, v):
        if vi != 0:
            lambdas.append(Ai/vi)

    lambdas = list(set([sp.simplify(x) for x in lambdas]))
    #print(lambdas)

    if len(lambdas) == 1:
        return True, lambdas[0]
    else:
        return False, lambdas

print("\nCOMPROBACIÓN DE AUTOESTADOS DE Jz")
print("=" * 40)

for v in lista_autoestados_normalizados:
    esJz, valJz = comprobar_autoestado(Jz, v)

    if esJz:
        print("\nEl autoestado:\n")
        sp.pprint(v)
        print(f"Es autovalor de Jz con valor = {valJz} ")
        print("-" * 40)
    else:
        print("No es autovalor de Jz")


COMPROBACIÓN DE AUTOESTADOS DE Jz
No es autovalor de Jz

El autoestado:

⎡        0         ⎤
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎢-0.577350269189626⎥
⎢                  ⎥
⎢0.816496580927726 ⎥
⎢                  ⎥
⎣        0         ⎦
Es autovalor de Jz con valor = 0.5*hbar 
----------------------------------------

El autoestado:

⎡1.0⎤
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎣ 0 ⎦
Es autovalor de Jz con valor = -1.5*hbar 
----------------------------------------

El autoestado:

⎡        0        ⎤
⎢                 ⎥
⎢0.577350269189626⎥
⎢                 ⎥
⎢0.816496580927726⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎣        0        ⎦
Es autovalor de Jz con valor = -0.5*hbar 
----------------------------------------
No es autovalor de Jz

El autoestado:

⎡ 0 ⎤
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎣1.0⎦
Es autovalor de Jz con

In [23]:
print("\nCOMPROBACIÓN DE AUTOESTADOS DE J2")
print("=" * 40)

for v in lista_autoestados_normalizados:
    esJ2, valJ2 = comprobar_autoestado(J2, v)

    if esJ2:
        print("\nEl autoestado:\n")
        sp.pprint(v)
        print(f"Es autovalor de J2 con valor = {valJ2} ")
        print("-" * 40)
    else:
        print("Tiene mas de una autovalor, por lo que no es autovalor de J2")
        print("\nEl autoestado:\n")
        sp.pprint(v)
        print(f"Los posibles autovalores son: {valJ2} ")
        print("-" * 40)



COMPROBACIÓN DE AUTOESTADOS DE J2
Tiene mas de una autovalor, por lo que no es autovalor de J2

El autoestado:

⎡        0         ⎤
⎢                  ⎥
⎢-0.816496580927726⎥
⎢                  ⎥
⎢0.577350269189626 ⎥
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎣        0         ⎦
Los posibles autovalores son: [hbar*(0.5 - 0.707106781186547*sqrt(2)), hbar*(1.5 - 1.4142135623731*sqrt(2))] 
----------------------------------------
Tiene mas de una autovalor, por lo que no es autovalor de J2

El autoestado:

⎡        0         ⎤
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎢-0.577350269189626⎥
⎢                  ⎥
⎢0.816496580927726 ⎥
⎢                  ⎥
⎣        0         ⎦
Los posibles autovalores son: [hbar*(1.5 - 1.41421356237309*sqrt(2)), hbar*(0.5 - 0.707106781186548*sqrt(2))] 
----------------------------------------

El autoestado:

⎡1.0⎤
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 0

Coeficientes de Clebsch-Gordan se utilizan cuando un sistema con operadores acoplados, en este caso vamos tener los operadores del momento angular $\hat{L}$ y $\hat{S}$ acoplados, por lo que $\hat{J} = \hat{L} + \hat{S}$ se conserva. Se puede construir una base de estados que sea simultaneamente autofuncines de los operadores $\hat{L}$ y $\hat{S}$:

$$
\ket{J, M} = \ket{j_1, m_1; j_2, m_2} =  \ket{j_1, m_1} \ket{j_2, m_2}
$$

Las cuales tiene dimención $(2 j_1 + 1)(2 j_2 + 1)$, tenemos que las dos descripciones son equivalentes y se pueden obtener por la transformación:

$$
\ket{J,M} = \sum_{j,m} C_{j_1, j_2, j; m_1, m_2, m} \ket{j_1,m_1; j_2, m_2}
$$

Donde:

$$
C_{j_1, j_2, j; m_1, m_2, m} = \braket{j_1,m_1; j_2, m_2 | j, m}
$$

Los coeficientes de Clebsch-Gordan desaparecen a no se que se cumpla la condicion triangular la cual es:

* $m = m_1 + m_2$
* $|j_1 - j_2| \leq j \leq |j_1 + j_2|$

In [39]:
valor_autoestado = lista_autoestados_normalizados[0]

print(f"{valor_autoestado[1]:.6f}")

-0.816497


In [40]:
valor_autoestado = lista_autoestados_normalizados[0]

print(f"{valor_autoestado[2]:.6f}")

0.577350
